In [6]:
import geemap
import ee
geemap.ee_initialize()

In [8]:
# only doing one simple work done by using geemap about ndvi map of chitwan districts
# import shape file get the chitwan district only
import geemap
import geopandas as gpd
all_district=r"C:\Users\Acer\Desktop\miniproject\data\data2\Districts.shp"
df=gpd.read_file(all_district)
chitwanonly=df[df["DIST_NAME"]== "Chitawan"]
chitwanonly
m=geemap.Map()
m.add_gdf(chitwanonly, layer_name="chitwan")

In [9]:
chitwanonly=chitwanonly.to_crs(epsg=4326)
chitwan_dist=geemap.gdf_to_ee(chitwanonly)

In [10]:
# compute ndvi for each image :

def add_ndvi(image):
    """Adds an NDVI band to an image."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def mask_clouds(image):
    """Masks clouds using the 'MSK_CLDPRB' band."""
    clouds = image.select('MSK_CLDPRB').gt(65) # Greater than 65% cloud probability
    return image.updateMask(clouds.Not())  # Mask out those pixels


In [11]:
chitwan_area=chitwan_dist.geometry()
def compute_ndvi_for_year(year, chitwan_area):
    """Computes NDVI composite for a given year."""

    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    # Load Sentinel-2 data, filter by date, region, and cloud cover.
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterDate(start_date, end_date)
                  .filterBounds(chitwan_area)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))) # 20% cloud cover or less

    # Apply cloud masking BEFORE NDVI calculation
    sentinel2_masked = sentinel2.map(mask_clouds)

    # Apply the NDVI function to the *masked* image collection.
    ndvi_collection = sentinel2_masked.map(add_ndvi)
    
    # Create a median composite from the NDVI collection.
    ndvi_composite = ndvi_collection.select('NDVI').median().clip(chitwan_area)
    composite_band_names = ndvi_composite.bandNames()
    print(f"Band names of NDVI composite for year {year}: {composite_band_names.getInfo()}")

    return ndvi_composite
years = range(2019,2025)  # 2018 to 2024

# OR Create an empty dictionary to store the yearly NDVI composites (keyed by year).
# yearly_ndvi_composites = {} #dictionary

#yearly_ndvi_composites = []  # You don't need this if you're only using the dictionary
yearly_ndvi_composites_dict = {}  # Initialize as an empty DICTIONARY!

# Iterate through the years and compute the NDVI composites
for year in years:
  print(f"Processing year: {year}")  # Optional: Print progress
  ndvi_composite = compute_ndvi_for_year(year, chitwan_area) 
  print(f"finish processing {year}") # Call your function

    # Append the result to the list
    #yearly_ndvi_composites.append(ndvi_composite) #error here, should remove this if using dictionary

    # OR Store the result in the dictionary, keyed by the year.
  yearly_ndvi_composites_dict[year] = ndvi_composite  #Correct way of assigning value to dictionary

Processing year: 2019
Band names of NDVI composite for year 2019: ['NDVI']
finish processing 2019
Processing year: 2020
Band names of NDVI composite for year 2020: ['NDVI']
finish processing 2020
Processing year: 2021
Band names of NDVI composite for year 2021: ['NDVI']
finish processing 2021
Processing year: 2022
Band names of NDVI composite for year 2022: ['NDVI']
finish processing 2022
Processing year: 2023
Band names of NDVI composite for year 2023: ['NDVI']
finish processing 2023
Processing year: 2024
Band names of NDVI composite for year 2024: ['NDVI']
finish processing 2024


In [12]:
def dict_to_imagecollection(ndvi_dict):
    images = []
    for year, img in ndvi_dict.items():
        # Assign time property so ts_inspector can plot by year
        img = img.set('system:time_start', ee.Date.fromYMD(year, 1, 1).millis())
        images.append(img)
    return ee.ImageCollection(images)

# Convert
ndvi_ic = dict_to_imagecollection(yearly_ndvi_composites_dict)


In [15]:
ndvi_vis_params = {
    'min': 0,
    'max': 1,
    'palette': [
        "#d73027", "#fdae61", "#fee08b",
        "#d9ef8b", "#1a9850", "#006837"
    ]
}
m.centerObject(chitwan_area, 10)

# Add Time Series Inspector
m.ts_inspector(
    left_ts=ndvi_ic,
    right_ts=ndvi_ic,
    left_names=[str(y) for y in years],
    right_names=[str(y) for y in years],
    left_vis=ndvi_vis_params,
    right_vis=ndvi_vis_params,
    width="80px",
)
m

Map(bottom=110461.0, center=[27.58469920634319, 84.43650321937417], controls=(WidgetControl(options=['position…

In [16]:
ndvi_legend={
    "Water / No vegetation": "#d73027",
    "Barren land / Built-up": "#fdae61",
    "Sparse vegetation": "#fee08b",
    "Moderate vegetation": "#d9ef8b",
    "Dense vegetation": "#1a9850",
    "Very dense vegetation": "#006837"
}
m.add_legend(title='NDVI Categories',
              legend_dict =ndvi_legend,
              position="bottomright")  
m.add("basemap_selector")